![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/MULTICLF_LITCOVID.ipynb)

## **Multilabel Classification For LitCovid**

## **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.0.2
Spark NLP_JSL Version : 5.0.2


# **multiclassifierdl_litcovid**





In [ ]:
text_list = [
"""Low level of plasminogen increases risk for mortality in COVID-19 patients. The pathophysiology of coronavirus disease 2019 (COVID-19), caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), and especially of its complications is still not fully understood. In fact, a very high number of patients with COVID-19 die because of thromboembolic causes. A role of plasminogen, as precursor of fibrinolysis, has been hypothesized. In this study, we aimed to investigate the association between plasminogen levels and COVID-19-related outcomes in a population of 55 infected Caucasian patients (mean age: 69.8 +/- 14.3, 41.8% female). Low levels of plasminogen were significantly associated with inflammatory markers (CRP, PCT, and IL-6), markers of coagulation (D-dimer, INR, and APTT), and markers of organ dysfunctions (high fasting blood glucose and decrease in the glomerular filtration rate). A multidimensional analysis model, including the correlation of the expression of coagulation with inflammatory parameters, indicated that plasminogen tended to cluster together with IL-6, hence suggesting a common pathway of activation during disease's complication. Moreover, low levels of plasminogen strongly correlated with mortality in COVID-19 patients even after multiple adjustments for presence of confounding. These data suggest that plasminogen may play a pivotal role in controlling the complex mechanisms beyond the COVID-19 complications, and may be useful both as biomarker for prognosis and for therapeutic target against this extremely aggressive infection.""",
"""Potential role for tissue factor in the pathogenesis of hypercoagulability associated with in COVID-19. In December 2019, a new and highly contagious infectious disease emerged in Wuhan, China. The etiologic agent was identified as a novel coronavirus, now known as Severe Acute Syndrome Coronavirus-2 (SARS-CoV-2). Recent research has revealed that virus entry takes place upon the union of the virus S surface protein with the type I transmembrane metallo-carboxypeptidase, angiotensin converting enzyme 2 (ACE-2) identified on epithelial cells of the host respiratory tract. Virus triggers the synthesis and release of pro-inflammatory cytokines, including IL-6 and TNF-alpha and also promotes downregulation of ACE-2, which promotes a concomitant increase in levels of angiotensin II (AT-II). Both TNF-alpha and AT-II have been implicated in promoting overexpression of tissue factor (TF) in platelets and macrophages. Additionally, the generation of antiphospholipid antibodies associated with COVID-19 may also promote an increase in TF. TF may be a critical mediator associated with the development of thrombotic phenomena in COVID-19, and should be a target for future study.""",
"""Estimates of serial interval for COVID-19: A systematic review and meta-analysis. Background: On 11(th) March 2020, the World Health Organization declared COVID-19 as Pandemic. The estimation of transmission dynamics in the initial days of the outbreak of any infectious disease is crucial to control its spread in a new area. The serial interval is one of the significant epidemiological measures that determine the spread of infectious disease. It is the time interval between the onset of symptoms in the primary and secondary case. Objective: The present study aimed at the qualitative and quantitative synthesis of the currently available evidence for the serial interval of COVID-19. Methodology: Data on serial intervals were extracted from 11 studies following a systematic review. A meta-analysis was performed to estimate the pooled estimate of the serial interval. The heterogeneity and bias in the included studies were tested by various statistical measures and tests, including I(2) statistic, Cochran's Q test, Egger's test, and Beggs's test. Result: The pooled estimate for the serial interval was 5.40 (5.19, 5.61) and 5.19 (4.37, 6.02) days by the fixed and random effects model, respectively. The heterogeneity between the studies was found to be 89.9% by I(2) statistic. There is no potential bias introduced in the meta-analysis due to small study effects. Conclusion: The present review provides sufficient evidence for the estimate of serial interval of COVID-19, which can help in understanding the epidemiology and transmission of the disease. The information on serial interval can be useful in developing various policies regarding contact tracing and monitoring community transmission of COVID-19.""",
"""Placental Pathology Findings during and after SARS-CoV-2 Infection: Features of Villitis and Malperfusion. Since the outbreak of coronavirus disease 2019 (COVID-19), there has been a debate whether pregnant women are at a specific risk for COVID-19 and whether it might be vertically transmittable through the placenta. We present a series of five placentas of SARS coronavirus 2 (SARS-CoV-2)-positive women who had been diagnosed with mild symptoms of COVID-19 or had been asymptomatic before birth. We provide a detailed histopathologic description of morphological changes accompanied by an analysis of presence of SARS-CoV-2 in the placental tissue. All placentas were term deliveries (40th and 41st gestational weeks). One SARS-CoV-2-positive patient presented with cough and dyspnoea. This placenta showed prominent lymphohistiocytic villitis and intervillositis and signs of maternal and foetal malperfusion. Viral RNA was present in both placenta tissue and the umbilical cord and could be visualized by in situ hybridization in the decidua. SARS-CoV-2 tests were negative at the time of delivery of 3/5 women, and their placentas did not show increased inflammatory infiltrates. Signs of maternal and/or foetal malperfusion were present in 100% and 40% of cases, respectively. There was no transplacental transmission to the infants. In our cohort, we can document different time points regarding SARS-CoV-2 infection. In acute COVID-19, prominent lymphohistiocytic villitis may occur and might potentially be attributable to SARS-CoV-2 infection of the placenta. Furthermore, there are histopathological signs of maternal and foetal malperfusion, which might have a relationship to an altered coagulative or microangiopathic state induced by SARS-CoV-2, yet this cannot be proven considering a plethora of confounding factors."""

]

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document", "token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings()\
    .setInputCols(["document", "word_embeddings"])\
    .setOutputCol("sentence_embeddings")\
    .setPoolingStrategy("AVERAGE")

multi_classifier_dl = MultiClassifierDLModel.pretrained("multiclassifierdl_litcovid", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")

pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
        multi_classifier_dl
    ])


pipeline_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = LightPipeline(pipeline_model)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
multiclassifierdl_litcovid download started this may take some time.
Approximate size to download 11.2 MB
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "class.result")
df = x.toDF('text', 'class')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+---------------------------------+
|                                                                                                text|                            class|
+----------------------------------------------------------------------------------------------------+---------------------------------+
|[Low level of plasminogen increases risk for mortality in COVID-19 patients. The pathophysiology ...|[Mechanism, Treatment, Diagnosis]|
|[Potential role for tissue factor in the pathogenesis of hypercoagulability associated with in CO...|           [Mechanism, Treatment]|
|[Estimates of serial interval for COVID-19: A systematic review and meta-analysis. Background: On...|                     [Prevention]|
|[Placental Pathology Findings during and after SARS-CoV-2 Infection: Features of Villitis and Mal...|                    [Case_Report]|
+----------------------------------------